In [ ]:
import os
import openai
import json

In [ ]:
import sys
print(sys.executable)

In [ ]:
SYSTEM_MESSAGE = """You are an AI assistant that is able to convert natural language into a properly formatted SQL query.

The table you will be querying is called "finances". Here is the schema of the table:
'finances': {'id': 'INTEGER', 'date': 'TEXT', 'revenue': 'REAL', 'expenses': 'REAL', 'profit': 'REAL'}

You must always output your answer in a JSON document with the following key-value pairs:
- "query": the SQL query that you generated
- "error": an error message if the query is invalid, or null if the query is valid

Output only the JSON document, nothing else.

Here is an example if how the output should look like:
{
    "query": "SELECT * FROM finances WHERE expenses > 1000",
    "error": null
}
"""

In [ ]:
user_message = "Give an example of an SQL statement"

messages = [
        {'role': 'system', 'content': SYSTEM_MESSAGE},
        {'role': 'user', 'content': f"{user_message}"}
    ]

In [ ]:

openai.api_type = "azure"
openai.api_base = "https://anforsbe-sweden-central.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.ChatCompletion.create(
  engine="gpt-4",
  messages = messages,
  temperature=0.0,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

In [ ]:
print(response)

In [ ]:
response = response.choices[0].message["content"]
response

In [ ]:
json_response = json.loads(response)
query = json_response['query']
query


# pandas AI with AzureOpenAI

*Working with Agent*

With the chat agent, you can engage in dynamic conversations where the agent retains context throughout the discussion. This enables you to have more interactive and meaningful exchanges.

Key Features

Context Retention: The agent remembers the conversation history, allowing for seamless, context-aware interactions.

Clarification Questions: You can use the clarification_questions method to request clarification on any aspect of the conversation. This helps ensure you fully understand the information provided.

Explanation: The explain method is available to obtain detailed explanations of how the agent arrived at a particular solution or response. It offers transparency and insights into the agent's decision-making process.

In [4]:
import os
import openai
from dotenv import load_dotenv

import pandas as pd
from pandasai import Agent
from pandasai.skills import skill
# from pandasai import SmartDataframe

from pandasai.llm.azure_openai import AzureOpenAI



In [ ]:
load_dotenv()

In [5]:
employees_data = {
    "EmployeeID": [1, 2, 3, 4, 5],
    "Name": ["John", "Emma", "Liam", "Olivia", "William"],
    "Department": ["HR", "Sales", "IT", "Marketing", "Finance"],
}

salaries_data = {
    "EmployeeID": [1, 2, 3, 4, 5],
    "Salary": [5000, 6000, 4500, 7000, 5500],
}

employees_df = pd.DataFrame(employees_data)
salaries_df = pd.DataFrame(salaries_data)

In [ ]:
azure_openai_api_key = os.getenv("OPENAI_API_KEY")
azure_openai_api_type = os.getenv("AZURE_OPENAI_API_TYPE")
azure_openai_api_base = os.getenv("AZURE_OPENAI_API_BASE")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name = os.getenv("AZURE_API_DEPLOYMENT_NAME")

In [ ]:
deployment_name

In [ ]:
llm = AzureOpenAI(api_token=azure_openai_api_key, 
            api_base=azure_openai_api_base,
            api_version=azure_openai_api_version,
            deployment_name="gpt-4-pandas-ai", 
            is_chat_model=True)

user_defined_path = "LLM_play\charts_saved"

# llm = AzureOpenAI(api_token=AZURE_OPENAI_API_KEY, api_base=openai_api_base,api_version=openai_api_version,deployment_name="gpt-4", is_chat_model=True)
#agent = Agent([employees_df, salaries_df], config={"llm": llm}, memory_size=10)

agent = Agent([employees_df, salaries_df], config={"llm": llm, "save_charts": True, "save_charts_path": user_defined_path}, memory_size=10)


In [ ]:
# Adding a custom skill
# Note: That the function doc string give more context to the model for use this skill
@skill
def plot_horisontal_bar_capital_names(name_lst: list[str], salaries: list[int]):
    """
    Displays the a horizontal bar chart having name on y axis and salaries on x axis where the names will be captial letters
    Args:
        name (list[str]): Employee name
        salaries (list[int]): Salaries
    """
    # plot bars

    import matplotlib.pyplot as plt

    new_lst = []

    for name in name_lst:
        new_lst.append(name.upper())

    new_lst

    plt.barh(new_lst, salaries)
    # plt.bar(new_lst, salaries_data["Salary"])
    plt.xlabel("Salary")
    plt.ylabel("EMPLOYEE NAMES")
    plt.title("Employee Salaries from a custom skill")
    plt.xticks(rotation=45)



In [ ]:
# If hte skill is already added it will show upan an error message if added again. Hence in a try excpet
try: 
    agent.add_skills(plot_horisontal_bar_capital_names)
except:
    pass

In [ ]:
# Chat with the agent
response = agent.chat("What is the highet salary?")
print(response)

In [ ]:
type(response)

In [ ]:
# Chat with the agent
response = agent.chat("Who gets paid the most?")
print(response)


In [ ]:
# Chat with the agent
response = agent.chat("Plot a pie chart of the department distribution")
print(type(response))
print(response)

In [ ]:
# Chat with the agent
response = agent.chat("Create a boxplot of the salaries")
print(response)

In [ ]:
# TODO save the image from pandas ai and then read it back 
# https://docs.streamlit.io/library/api-reference/media/st.image
# https://docs.pandas-ai.com/en/latest/examples/#saving-plots-with-user-defined-path
# Create a function to retreive an image and then remove it and then use streamlit to show it

# Read image

In [ ]:
from PIL import Image
import os

In [ ]:
def get_first_image_in_folder(folder_path):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Filter only image files (you can customize this condition)
    image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    if not image_files:
        print("No image files found in the folder.")
        return None

    # Get the first image file
    first_image_path = os.path.join(folder_path, image_files[0])

    # Open the image using Pillow
    image = Image.open(first_image_path)

    return image



In [ ]:
# Example usage
folder_path = "LLM_play\charts_saved"
first_image = get_first_image_in_folder(folder_path)

if first_image:
    first_image.show()  # Opens the image using the default image viewer

In [ ]:
type(first_image)

In [ ]:
def remove_png_files(folder_path):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Filter only PNG files
    png_files = [file for file in files if file.lower().endswith('.png')]

    if not png_files:
        print("No PNG files found in the folder.")
        return

    # Remove each PNG file
    for png_file in png_files:
        file_path = os.path.join(folder_path, png_file)
        os.remove(file_path)
        print(f"Removed: {file_path}")

In [ ]:
# Example usage:
folder_path = "LLM_play\charts_saved"
remove_png_files(folder_path)

In [ ]:
from main_app_folder.support_functions import SupportMainApp

In [ ]:
from main_app_folder.support_functions import SupportMainApp

In [ ]:
image = SupportMainApp.get_first_png_image_in_folder(folder_path="temp_charts")

In [ ]:
type(image)

In [ ]:
SupportMainApp.remove_png_files(folder_path="temp_charts")

In [1]:
from pandas_azure_openai import PandasAIClass

In [7]:
agent_no_chart = PandasAIClass.return_pandas_agent_no_chart(subset_data=salaries_df)

In [1]:
response_image = None

In [2]:
if response_image:
    print("hi")

In [8]:
response_image = "image"

In [9]:
response_image == None

False

In [2]:
import os

In [3]:
os.remove("temp_chart.png")